In [4]:
import os
checkpoint_id = "2faf2a37-7cc4-4b2c-962d-9df04685d1cd"
os.chdir(f"/home/christophstach/Code/HTW/det-gan-research/checkpoints/{checkpoint_id}/code/")

In [5]:
import cv2
import math
import torch
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import scipy.ndimage
import numpy as np
import seaborn as sns
import utils

from ipywidgets import Video
from tqdm.notebook import tqdm
from determined.experimental import Checkpoint
from models import MsgGenerator
from msg_gan import MsgGANTrail
from metrics import InceptionScore
from torchvision.utils import make_grid

from distributions import TruncatedNormal

toPILImage = transforms.ToPILImage()

In [6]:
trail = Checkpoint.load_from_path("../", map_location=torch.device('cpu'))
generator = trail.generator.eval()

RuntimeError: Error(s) in loading state_dict for MsgDiscriminator:
	Unexpected key(s) in state_dict: "from_rgb_combiners.0.conv.weight", "from_rgb_combiners.0.conv.bias", "from_rgb_combiners.1.conv.weight", "from_rgb_combiners.1.conv.bias", "from_rgb_combiners.2.conv.weight", "from_rgb_combiners.2.conv.bias", "from_rgb_combiners.3.conv.weight", "from_rgb_combiners.3.conv.bias". 
	size mismatch for blocks.1.conv1.bias: copying a param with shape torch.Size([80]) from checkpoint, the shape in current model is torch.Size([40]).
	size mismatch for blocks.1.conv1.weight_orig: copying a param with shape torch.Size([80, 80, 3, 3]) from checkpoint, the shape in current model is torch.Size([40, 40, 3, 3]).
	size mismatch for blocks.1.conv1.weight_u: copying a param with shape torch.Size([80]) from checkpoint, the shape in current model is torch.Size([40]).
	size mismatch for blocks.1.conv1.weight_v: copying a param with shape torch.Size([720]) from checkpoint, the shape in current model is torc

In [ ]:
num_images  = 6
truncations = (-2.25, 2.25)
noise_size  = trail.latent_dimension
num_grid_images = 8 ** 2

z = utils.sample_noise(num_images, noise_size, truncations=truncations)
dist = utils.sample_noise(1, 100000, truncations=truncations).squeeze()
inception_z = utils.sample_noise(128, noise_size, truncations=truncations)
grid_z = utils.sample_noise(num_grid_images, noise_size, truncations=truncations)
    
ax = sns.displot(dist, bins=np.arange(-4, 4, 0.25), kde=True)
plt.show()

In [ ]:
images, w = generator(z)
biggest_size = images[-1].shape[-1]
num_resolutions = int(math.log2(biggest_size))

resolutions, x = generator(z)
fig, axes = plt.subplots(nrows=num_images, ncols=num_resolutions - 1, dpi=200, figsize=(6, 6))

for col, resolution in enumerate(resolutions):
    resolution = utils.shift_image_range(resolution)
    for row, tensor in enumerate(resolution):        
        image = toPILImage(tensor.cpu())
        
        axes[row, col].imshow(image)
        axes[row, col].get_xaxis().set_visible(False)
        axes[row, col].get_yaxis().set_visible(False)

In [ ]:
grid_images, _ = generator(grid_z)
grid_images = grid_images[-1]
grid_images = utils.shift_image_range(grid_images)
grid = make_grid(grid_images, nrow=int(math.sqrt(num_grid_images)))

fig, ax = plt.subplots(figsize=(16, 16), dpi=200)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)
ax.imshow(grid.permute(1, 2, 0))

In [ ]:
eval_model, resize_to, num_classes = utils.create_evaluation_model("default")
inception_score = InceptionScore(eval_model, resize_to, num_classes)

images, _ = generator(inception_z)
inception_score.images = images[-1]
inception_score()

In [ ]:
out_dir = "temp"
os.makedirs(out_dir, exist_ok=True)
video_path = 'temp/interpolation.mp4'


time = 30
fps = 30
smoothing = 1.25
total_frames = time * fps

fourcc = cv2.VideoWriter_fourcc(*'VP90')
video = cv2.VideoWriter(video_path, fourcc=fourcc, fps=fps, frameSize=(biggest_size, biggest_size))

all_z = utils.sample_noise(total_frames, noise_size, truncations=truncations)
all_z = scipy.ndimage.gaussian_filter(all_z, [smoothing * fps, 0])
all_z = torch.from_numpy(all_z)

for idx, z in tqdm(enumerate(all_z), desc="Generating video", total=total_frames):
    frame = idx + 1
    z.unsqueeze_(dim=0)
    tensors, w = generator(z)
    tensor = tensors[-1].squeeze()
    tensor = utils.shift_image_range(tensor)
    
    image = toPILImage(tensor.cpu())
    image = np.array(image)
    image = image[:, :, ::-1].copy() 

    video.write(image)
    
video.release()
Video.from_file(video_path, play=True)